In [7]:
import cv2
import numpy as np
from pathlib import Path
import os
from glob import glob

def save_change_detection(src_dir, output_dir, downscale_factor=4, prefix=""):

    os.makedirs(output_dir, exist_ok=True)

    image_pairs = []
    for img1_path in sorted(glob(os.path.join(src_dir, "initial_paired_frame_*.jpg"))):
        basename = os.path.basename(img1_path).replace("initial_paired_", "")
        img2_path = os.path.join(src_dir, f"{prefix}{basename}")
        if os.path.exists(img2_path):
            image_pairs.append((img1_path, img2_path))
        else:
            print(f"⚠️ Not found: {img2_path}")

    for idx, (img1_path, img2_path) in enumerate(image_pairs):

        img1 = cv2.imread(img1_path)
        img2 = cv2.imread(img2_path)

        if img1 is None or img2 is None:
            print(f"❌ Failed to read images: {img1_path}, {img2_path}")
            continue

        # Resize both images
        height, width = img1.shape[:2]
        # new_size = (width // downscale_factor, height // downscale_factor)
        # img1_resized = cv2.resize(img1, new_size)
        # img2_resized = cv2.resize(img2, new_size)
        img1_resized = img1
        img2_resized = img2

        # Convert to grayscale
        gray1 = cv2.cvtColor(img1_resized, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(img2_resized, cv2.COLOR_BGR2GRAY)

        # Absolute difference
        difference = cv2.absdiff(gray1, gray2)

        # Threshold
        _, thresholded = cv2.threshold(difference, 80, 255, cv2.THRESH_BINARY)

        # # Morphology
        # kernel_erode = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        # eroded = cv2.erode(thresholded, kernel_erode, iterations=1)

        # kernel_dilate = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
        # dilated = cv2.dilate(eroded, kernel_dilate, iterations=3)

        # Extract frame index from filename
        original_basename = os.path.basename(img1_path)
        index_str = original_basename.replace("initial_paired_frame_", "").replace(".jpg", "")
        frame_filename = f"frame_{index_str}.png"
        output_path = Path(output_dir) / frame_filename

        # Save only the final morphology result
        cv2.imwrite(str(output_path), thresholded)
        print(f"✅ Saved: {output_path.name}")

# 실행 경로 설정
base_dir = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Duraemidam_ChangeDetection_workspace/trial4/CD_PAIR/test2"
src_dir = os.path.join(base_dir, "nerf_cd_pair/rectified")
output_dir = os.path.join(base_dir, "evaluate/diff_based_method")

save_change_detection(src_dir, output_dir, downscale_factor=4, prefix="new_")


✅ Saved: frame_00005.png
✅ Saved: frame_00007.png
✅ Saved: frame_00011.png
✅ Saved: frame_00021.png
✅ Saved: frame_00025.png


In [10]:
import cv2
import numpy as np
import os
import glob
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score

def load_binary_mask(path, downscale_factor=4):
    """
    Load a grayscale mask image, downscale, and normalize to 0/1 binary mask.
    Also prints whether it's 0/1 or 0/255.
    """
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise FileNotFoundError(f"Could not read image: {path}")    

    # Resize (downscale)
    height, width = mask.shape
    resized = cv2.resize(mask, (width // downscale_factor, height // downscale_factor), interpolation=cv2.INTER_NEAREST)

    # ✅ 강제 이진화 (128 기준 threshold)
    _, binary_mask = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY)

    # Determine if mask is 0/1 or 0/255
    unique_values = np.unique(binary_mask)
    print(f"[{os.path.basename(path)}] Unique values: {unique_values}")

    

    if set(unique_values.tolist()).issubset({0, 1}):
        binary = binary_mask
    elif set(unique_values.tolist()).issubset({0, 255}):
        binary = binary_mask // 255
    else:
        raise ValueError(f"Unexpected mask values in {path}: {unique_values}")

    return binary

def evaluate_masks(ground_truth_dir, predicted_dir, downscale_factor=4):
    # 평가 지표 저장용 리스트
    accuracies, precisions, recalls, f1_scores, ious = [], [], [], [], []

    gt_paths = sorted(glob.glob(os.path.join(ground_truth_dir, '*.jpg')) +
                      glob.glob(os.path.join(ground_truth_dir, '*.png')))
    pred_paths = sorted(glob.glob(os.path.join(predicted_dir, '*.jpg')) +
                        glob.glob(os.path.join(predicted_dir, '*.png')))

    assert len(gt_paths) == len(pred_paths), "GT and prediction count mismatch!"

    for gt_path, pred_path in zip(gt_paths, pred_paths):
        gt_mask = load_binary_mask(gt_path, downscale_factor)
        pred_mask = load_binary_mask(pred_path, downscale_factor)

        # Flatten
        gt_flat = gt_mask.flatten()
        pred_flat = pred_mask.flatten()

        # Metric 계산
        accuracies.append(accuracy_score(gt_flat, pred_flat))
        precisions.append(precision_score(gt_flat, pred_flat, zero_division=1))
        recalls.append(recall_score(gt_flat, pred_flat, zero_division=1))
        f1_scores.append(f1_score(gt_flat, pred_flat, zero_division=1))
        ious.append(jaccard_score(gt_flat, pred_flat, zero_division=1))

    # 평균 출력
    print("\n🔍 Evaluation Results (Downscaled by factor of {})".format(downscale_factor))
    print(f"Mean Accuracy : {np.mean(accuracies):.4f}")
    print(f"Mean Precision: {np.mean(precisions):.4f}")
    print(f"Mean Recall   : {np.mean(recalls):.4f}")
    print(f"Mean F1 Score : {np.mean(f1_scores):.4f}")
    print(f"Mean IoU      : {np.mean(ious):.4f}")

# 사용 예시
base_dir = '/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/CD_PAIR/test2'
ground_truth_dir = os.path.join(base_dir, 'outputs/GT/train/masks/integrated_masks/binary')
baseline_dir = os.path.join(base_dir,'evaluate/diff_based_method' )
predicted_dir = os.path.join(base_dir,'outputs/pred' )

evaluate_masks(ground_truth_dir, predicted_dir, downscale_factor=1)
evaluate_masks(ground_truth_dir, baseline_dir, downscale_factor=1)


[frame_00007.png] Unique values: [  0 255]
[frame_00007.jpg] Unique values: [  0 255]
[frame_00050.png] Unique values: [  0 255]
[frame_00050.jpg] Unique values: [  0 255]
[frame_00094.png] Unique values: [  0 255]
[frame_00094.jpg] Unique values: [  0 255]
[frame_00142.png] Unique values: [  0 255]
[frame_00142.jpg] Unique values: [  0 255]

🔍 Evaluation Results (Downscaled by factor of 1)
Mean Accuracy : 0.9828
Mean Precision: 0.9531
Mean Recall   : 0.6615
Mean F1 Score : 0.7599
Mean IoU      : 0.6327
[frame_00007.png] Unique values: [  0 255]
[frame_00007.png] Unique values: [  0 255]
[frame_00050.png] Unique values: [  0 255]
[frame_00050.png] Unique values: [  0 255]
[frame_00094.png] Unique values: [  0 255]
[frame_00094.png] Unique values: [  0 255]
[frame_00142.png] Unique values: [  0 255]
[frame_00142.png] Unique values: [  0 255]

🔍 Evaluation Results (Downscaled by factor of 1)
Mean Accuracy : 0.9348
Mean Precision: 0.3211
Mean Recall   : 0.3437
Mean F1 Score : 0.3283
Mean 